<a href="https://colab.research.google.com/github/dbremont/Notas/blob/main/Libros/Computacion/xv6_a_simple%2C_Unix_like_teaching_operating_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [xv6: a simple, Unix-like teaching operating system](https://pdos.csail.mit.edu/6.S081/2020/xv6/book-riscv-rev1.pdf)

- [QEMU setup instructions](http://www.cs.columbia.edu/~junfeng/11sp-w4118/tools.html)

## Operative system interfaces

-  The job of the operating system is to share a computer among multiple programs and to provide a more useful set of services than the hardware alone supports. 

- An **os** abstract the low-level hardware, so for example a word processor does not need to know which type of disk hardware is being used.

- An **os**  provides servicies to users programs through an **interface**. Designign a good interface is dificult.

- Good interfaces:
  - Simples,
  - Narrow -> makes easier to get the implementation right.

- In order to design a good **os** interface one has to balance the services that the interface provides vs its simplicify + narrowness.
  - A good way to find this balance is to provides a few mechanisms that can be combine to providate much generality.

- **kernel** a special program that provides services to other programs.
- Each running program is called a **process**, has memory containing instructions, data, and a stack. The instructions implement the programs computations.

- The computations alters the data.

- The stack organizes the programs's procedures calls.

- When a proces needs to invoke a kernel service, it invokes a **system call**, one of the calls in the operating system's interface. The system enters  the kernel; the kernel performs the service and returns. Thus a process alternates between executing in **user space** and **kernel space**.

- The kernel uses the hardware protectiosn mechanims provided by the **CPU**,  to ensure that each process  executing in user space can access only its own memory.

- The kernel executes with the hardware privileges.

- When a user program invokes a system call, the hardware raises the privilege level and starts executing a pre-arranged function in the kernel.

- The collection of system calls that a kernel  provides is the interface that users programs see.


**Processes and Memory**

**Preguntas, Ejercicios**

  - see the shell in user/sh.c
  - hardware privilemod
  - userpace vs kernel space
  - system calls
  - what is an os
  - trade offs in designing an os interface?
  - what are the componetns of a process?

## Page tables

1.  Parse RISC-V’s device tree to find the amount of physical memory the computer has.

2. Write a user program that grows its address space by one byte by calling `sbrk(1)`. Run
the program and investigate the page table for the program before the call to `sbrk` and after
the call to `sbrk`. How much space has the kernel allocated? What does the PTE for the new
memory contain?

3. Modify xv6 to use super pages for the kernel.

4. Modify xv6 so that when a user program dereferences a null pointer, it will receive an exception. That is, modify xv6 so that virtual address 0 isn’t mapped for user programs.

5. Unix implementations of `exec` traditionally include special handling for shell scripts. If the
file to execute begins with the text `#!`, then the first line is taken to be a program to run to
interpret the file. For example, if exec is called to run myprog arg1 and myprog ’s first
line is `#!/interp`, then exec runs `/interp` with command line `/interp myprog arg1`.
Implement support for this convention in xv6.

6. Implement address space randomization for the kernel

## Traps and system calls

1. The functions `copyin` and copyinstr walk the user page table in software. Set up
the kernel page table so that the kernel has the user program mapped, and `copyin` and
`copyinstr` can use `memcpy` to copy system call arguments into kernel space, relying on
the hardware to do the page table walk

2. Implement lazy memory allocation

3. Implement COW fork

## Interrupts and device drivers

1. Modify `uart.c` to not use interrupts at all. You may need to modify `console.c` as well.

2. Add a driver for an Ethernet card

## Locking

1. Comment out the calls to `acquire` and `release` in kalloc (kernel/kalloc.c:69). This seems
like it should cause problems for kernel code that calls `kalloc`; what symptoms do you
expect to see? When you run xv6, do you see these symptoms? How about when running
`usertests`? If you don’t see a problem, why not? See if you can provoke a problem by
inserting dummy loops into the critical section of `kalloc`.

2. Suppose that you instead commented out the locking in `kfree` (after restoring locking
in `kalloc`). What might now go wrong? Is lack of locks in `kfree` less harmful than in
`kalloc`?

3. If two CPUs call `kalloc` at the same time, one will have to wait for the other, which is bad
for performance. Modify `kalloc.c` to have more parallelism, so that simultaneous calls to
`kalloc` from different CPUs can proceed without waiting for each other.

4. Write a parallel program using POSIX threads, which is supported on most operating systems. For example, implement a parallel hash table and measure if the number of puts/gets
scales with increasing number of cores.

5. Implement a subset of Pthreads in xv6. That is, implement a user-level thread library so that
a user process can have more than 1 thread and arrange that these threads can run in parallel
on different CPUs. Come up with a design that correctly handles a thread making a blocking
system call and changing its shared address space.

## Scheduling

1. Sleep has to check lk != &p->lock to avoid a deadlock (kernel/proc.c:558-561). Suppose
the special case were eliminated by replacing
```c
if(lk != &p->lock){
  acquire(&p->lock);
  release(lk);
}
```

  with

  ```c
release(lk);
acquire(&p->lock);
```
Doing this would break sleep. How?

2. Most process cleanup could be done by either `exit` or `wait`. It turns out that exit must be
the one to close the open files. Why? The answer involves pipes.

3. Implement semaphores in xv6 without using `sleep` and `wakeup` (but it is OK to use spin
locks). Replace the uses of sleep and wakeup in xv6 with semaphores. Judge the result.

4. Fix the race mentioned above between `kill` and `sleep`, so that a `kill` that occurs after
the victim’s sleep loop checks `p->killed` but before it calls `sleep` results in the victim
abandoning the current system call.

5. Design a plan so that every sleep loop checks `p->killed` so that, for example, a process
that is in the virtio driver can return quickly from the while loop if it is killed by another
process.

6. Modify xv6 to use only one context switch when switching from one process’s kernel thread
to another, rather than switching through the scheduler thread. The yielding thread will need
to select the next thread itself and call `swtch`. The challenges will be to prevent multiple
cores from executing the same thread accidentally; to get the locking right; and to avoid
deadlocks.

7. Modify xv6’s `scheduler` to use the RISC-V `WFI` (wait for interrupt) instruction when no
processes are runnable. Try to ensure that, any time there are runnable processes waiting to
run, no cores are pausing in `WFI`.

8. The lock `p->lock` protects many invariants, and when looking at a particular piece of xv6
code that is protected by `p->lock`, it can be difficult to figure out which invariant is being
enforced. Design a plan that is more clean by splitting `p->lock` into several locks.

## File system

1. Why panic in `balloc` ? Can xv6 recover?

2. Why panic in `1ialloc` ? Can xv6 recover?

3. Why doesn’t `filealloc` panic when it runs out of files? Why is this more common and
therefore worth handling?

4. Suppose the file corresponding to ip gets unlinked by another process between `sys_link`
’s calls to `iunlock(ip)` and `dirlink`. Will the link be created correctly? Why or why not?

5. `create` makes four function calls (one to `ialloc` and three to `dirlink`) that it requires
to succeed. If any doesn’t, create calls panic. Why is this acceptable? Why can’t any of
those four calls fail?

6. `sys_chdir` calls `iunlock(ip)` before `iput(cp->cwd)`, which might try to lock `cp->cwd`,
yet postponing `iunlock(ip)` until after the iput would not cause deadlocks. Why not?

7. Implement the `lseek` system call. Supporting `lseek` will also require that you modify
filewrite to fill holes in the file with zero if `lseek` sets off beyond `f->ip->size`.

8. Add `O_TRUNC` and `O_APPEND` to open, so that `>` and `>>` operators work in the shell.

9. Modify the file system to support symbolic links.

## Concurrency revisited

1. Modify xv6’s pipe implementation to allow a read and a write to the same pipe to proceed in
parallel on different cores.

2. Modify xv6’s `scheduler()` to reduce lock contention when different cores are looking
for runnable processes at the same time.

3. Eliminate some of the serialization in xv6’s `fork()`